In [2]:
import geopandas as gpd
import pandas as pd

# Capturing Stops in Subway Stations and Major Intersections in QGIS

In [30]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
station_capture = gpd.read_file(simplify_dir + 'station_capture.geojson')
stops = pd.read_csv(gtfs_dir + 'stops.txt')

In [4]:
stops_1 = stops[~stops['stop_id'].isin(list(station_capture['stop_id']))]
stops_1.to_csv(simplify_dir + 'stops_nostations.csv', index = False)

In [7]:
major_capture = gpd.read_file(simplify_dir + 'major_capture.geojson')

In [8]:
stops_2 = stops_1[~stops_1['stop_id'].isin(list(major_capture['stop_id'].dropna()))]
stops_2.to_csv(simplify_dir + 'stops_no-stn-maj.csv', index = False)

# Finding Intersections Between Major Arterial and Minor Arterial

In [9]:
centreline = gpd.read_file('GIS/centreline.geojson')

In [10]:
intersections = gpd.read_file('GIS/major_minor_int.geojson')

In [11]:
intersections = intersections[['INT_ID', 'CLASSIFI6', 'INTERSEC5']]
intersections = intersections[intersections['CLASSIFI6'].isin(['MNRML', 'MNRML'])]

In [12]:
centreline = centreline[['CENTRELINE_ID', 'LINEAR_NAME_FULL', 'FROM_INTERSECTION_ID', 'TO_INTERSECTION_ID', 'FEATURE_CODE_DESC']]

In [13]:
centreline

,CENTRELINE_ID,LINEAR_NAME_FULL,FROM_INTERSECTION_ID,TO_INTERSECTION_ID,FEATURE_CODE_DESC
0,14646677,Antibes Dr,14646675,14646668,Collector
1,14257954,427 X S Gardiner X E Ramp,13470108,14257956,Expressway Ramp
2,20079785,Planning Boundary,20079775,13467222,Geostatistical line
3,20054934,Ellsworth Ave,20054935,20054926,Local
4,13969361,Keane Ave,13467359,13969362,Local
...,...,...,...,...,...
70231,30117986,Duncan Creek Trl,30006265,30110274,Trail
70232,30118134,Planning Boundary,30118132,30066723,Geostatistical line
70233,30105976,Gus Harris Trl,30001684,30069328,Trail
70234,30122184,Tommy Thompson Pk,13468364,30122183,Trail


In [14]:
centreline_maj = centreline[centreline['FEATURE_CODE_DESC'].isin(['Major Arterial', 'Minor Arterial'])]
centreline_maj

,CENTRELINE_ID,LINEAR_NAME_FULL,FROM_INTERSECTION_ID,TO_INTERSECTION_ID,FEATURE_CODE_DESC
6,14204002,Dundas St W,14204003,13467775,Major Arterial
15,14042202,Runnymede Rd,13465261,14042204,Minor Arterial
21,14068813,Rogers Rd,14068814,13461319,Minor Arterial
23,14063593,Dufferin St,14063594,14015767,Major Arterial
30,20062300,Jarvis St,20062301,13464792,Major Arterial
...,...,...,...,...,...
69998,20034591,Morningside Ave,14606950,14256135,Minor Arterial
70106,14659568,Finch Ave W,13451305,14659570,Major Arterial
70112,10536929,Oakdale Rd,13451142,13449708,Minor Arterial
70139,30020939,Norfinch Dr,30020937,13448222,Minor Arterial


In [15]:
intersections = intersections.drop_duplicates()

In [16]:
int_maj_min = centreline_maj.merge(intersections, right_on = ['INT_ID'], left_on = ['FROM_INTERSECTION_ID']).append(
    (centreline_maj.merge(intersections, right_on = ['INT_ID'], left_on = ['TO_INTERSECTION_ID'])))
int_maj_min

,CENTRELINE_ID,LINEAR_NAME_FULL,FROM_INTERSECTION_ID,TO_INTERSECTION_ID,FEATURE_CODE_DESC,INT_ID,CLASSIFI6,INTERSEC5
0,105089,Port Union Rd,13443463,13443458,Minor Arterial,13443463,MNRML,HWY 401 / Port Union Rd / Port Union S 401 C E...
1,20054575,Bathurst St,13468510,20054577,Major Arterial,13468510,MNRML,Bathurst St
2,442205,Wilson Heights Blvd,13451659,13451647,Minor Arterial,13451659,MNRML,Wilson Heights Blvd / Allen N Wilson Hs N Ramp
3,30095351,King St W,13468021,30095350,Major Arterial,13468021,MNRML,King St W / Crawford St
4,445824,Weston Rd,13456378,13456365,Major Arterial,13456378,MNRML,Weston Rd / Weston Rd S 401 C E Ramp
...,...,...,...,...,...,...,...,...
314,907171,Finch Ave W,13453152,13453184,Major Arterial,13453184,MNRML,Finch Ave W / Finch W 427 N Ramp / Humber Rive...
315,909596,Carlingview Dr,13462783,13462101,Major Arterial,13462101,MNRML,Carlingview Dr / Meteor Dr
316,1147157,The Queensway,13468360,13468559,Major Arterial,13468559,MNRML,The Queensway / Ellis Ave
317,4241728,Middlefield Rd,13443127,13442710,Minor Arterial,13442710,MNRML,Middlefield Rd / Scottfield Dr


In [17]:
int_maj_min = int_maj_min[['INT_ID', 'FEATURE_CODE_DESC']]
int_maj_min

,INT_ID,FEATURE_CODE_DESC
0,13443463,Minor Arterial
1,13468510,Major Arterial
2,13451659,Minor Arterial
3,13468021,Major Arterial
4,13456378,Major Arterial
...,...,...
314,13453184,Major Arterial
315,13462101,Major Arterial
316,13468559,Major Arterial
317,13442710,Minor Arterial


In [32]:
int_maj_min[int_maj_min['INT_ID'] ==  13460559]

,INT_ID,FEATURE_CODE_DESC


In [18]:
int_maj_min = pd.get_dummies(int_maj_min, columns = ['FEATURE_CODE_DESC']).groupby('INT_ID').sum().reset_index()

In [19]:
int_maj_min = int_maj_min[(int_maj_min['FEATURE_CODE_DESC_Major Arterial']>0) & 
            (int_maj_min['FEATURE_CODE_DESC_Minor Arterial']>0)]

In [20]:
intersections_geo = gpd.read_file('/Volumes/Data2/RST/notebook/GIS/intersection-file-wgs84/CENTRELINE_INTERSECTION_WGS84.shp')

In [21]:
int_maj_min = intersections_geo[intersections_geo['INT_ID'].isin(list(int_maj_min['INT_ID']))]
int_maj_min = int_maj_min.to_crs(epsg = '32617')

In [37]:
int_maj_min.to_file(simplify_dir + 'int_maj_min.geojson', driver = 'GeoJSON')

## Major/Minor Capture

In [15]:
maj_min_capture = gpd.read_file(simplify_dir + 'int_maj_min_capture.geojson')

In [16]:
stops_3 = stops_2[~stops_2['stop_id'].isin(list(maj_min_capture['stop_id'].dropna()))]
stops_3.to_csv(simplify_dir + 'stops_no-stn-maj-majmin.csv', index = False)

In [17]:
stops_3

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding
2,265,1871,DAVISVILLE AVE AT CLEVELAND ST,NaN,43.702096,-79.378146,NaN,NaN,NaN,NaN,1.0
3,266,11700,DISCO RD AT ATTWELL DR,NaN,43.701331,-79.594777,NaN,NaN,NaN,NaN,1.0
4,267,3478,DISCO RD AT ATTWELL DR,NaN,43.701057,-79.595799,NaN,NaN,NaN,NaN,1.0
5,268,3479,DISCO RD AT CARLINGVIEW DR EAST SIDE,NaN,43.699820,-79.600767,NaN,NaN,NaN,NaN,1.0
6,269,3480,DISCO RD AT CARLINGVIEW DR,NaN,43.699893,-79.601112,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
10470,15365,S_SUKI,KING & SUMACH,NaN,43.655575,-79.358402,NaN,NaN,NaN,NaN,2.0
10471,15370,15526,DON MILLS RD AT BARBER GREEN SOUTH SIDE,NaN,43.727325,-79.340877,NaN,NaN,NaN,NaN,1.0
10476,15377,S_FIBA,FINCH AVE E AT BALDOON RD,NaN,43.815237,-79.232350,NaN,NaN,NaN,NaN,2.0
10477,15378,S_YKYK,YORK UNIVERSITY BUSWAY AT YORK BLVD,NaN,43.774503,-79.496783,NaN,NaN,NaN,NaN,2.0


# Finding Intersections Between Minor Arterials

In [22]:
centreline_min_min = centreline[centreline['FEATURE_CODE_DESC'].isin(['Minor Arterial'])]
int_min_min = centreline_min_min.merge(intersections, right_on = ['INT_ID'], left_on = ['FROM_INTERSECTION_ID']).append(
    (centreline_min_min.merge(intersections, right_on = ['INT_ID'], left_on = ['TO_INTERSECTION_ID'])))
int_min_min = int_min_min[['INT_ID', 'FEATURE_CODE_DESC']]
int_min_min = pd.get_dummies(int_min_min, columns = ['FEATURE_CODE_DESC']).groupby('INT_ID').sum().reset_index()
int_min_min = int_min_min[(int_min_min['FEATURE_CODE_DESC_Minor Arterial']>2)]
int_min_min

,INT_ID,FEATURE_CODE_DESC_Minor Arterial
53,13462293,3
81,13466914,3
83,13467485,3
88,13468181,3
99,14025083,3


In [23]:
int_min_min = intersections_geo[intersections_geo['INT_ID'].isin(list(int_min_min['INT_ID']))]
int_min_min = int_min_min[~int_min_min['INT_ID'].isin(list(int_maj_min['INT_ID']))]
int_min_min = int_min_min.to_crs(epsg = '32617')
int_min_min

,INT_ID,ELEV_ID,INTERSEC5,CLASSIFI6,CLASSIFI7,NUM_ELEV,ELEVATIO9,ELEVATIO10,ELEV_LEVEL,ELEVATION,ELEVATIO13,HEIGHT_R14,HEIGHT_R15,X,Y,LONGITUDE,LATITUDE,OBJECTID,geometry
15731,13468181,41634,York St / Queens Quay W,MNRML,Minor-Multi Level,2,509200,Pseudo,9,0.0,None,0.0,None,314449.898,4833120.687,-79.380200,43.639964,13017.0,POINT (630649.629 4833160.367)
15732,13468181,14720,York St / Queens Quay W,MNRML,Minor-Multi Level,2,501300,Minor,0,0.0,None,0.0,None,314449.898,4833120.687,-79.380200,43.639964,13018.0,POINT (630649.629 4833160.367)
16776,13462293,42034,Spadina Rd / Davenport Rd,MNRML,Minor-Multi Level,2,509100,Cul de sac,9,0.0,None,0.0,None,312191.754,4837278.651,-79.408131,43.677417,5460.0,POINT (628316.932 4837276.402)
16777,13462293,7114,Spadina Rd / Davenport Rd,MNRML,Minor-Multi Level,2,501300,Minor,0,0.0,None,0.0,None,312191.754,4837278.651,-79.408131,43.677417,5461.0,POINT (628316.932 4837276.402)
26661,13466914,8414,Front St E / Farquhars Lane,MNRML,Minor-Multi Level,2,501300,Minor,0,0.0,None,0.0,None,315004.527,4834089.589,-79.373307,43.648678,48815.0,POINT (631186.635 4834139.015)
26662,13466914,39856,Front St E / Farquhars Lane,MNRML,Minor-Multi Level,2,509200,Pseudo,9,0.0,None,0.0,None,315004.527,4834089.589,-79.373307,43.648678,48816.0,POINT (631186.635 4834139.015)
26917,13467485,37496,Queens Quay E / Lower Sherbourne St,MNRML,Minor-Multi Level,2,509100,Cul de sac,9,0.0,None,0.0,None,315641.194,4833702.294,-79.365422,43.645182,40571.0,POINT (631830.134 4833763.313)
26918,13467485,26502,Queens Quay E / Lower Sherbourne St,MNRML,Minor-Multi Level,2,501300,Minor,0,0.0,None,0.0,None,315641.194,4833702.294,-79.365422,43.645182,40572.0,POINT (631830.134 4833763.313)
28908,14025083,40831,Parliament St,MNRML,Minor-Multi Level,2,509100,Cul de sac,9,0.0,None,0.0,None,315195.161,4836623.547,-79.370895,43.671484,12101.0,POINT (631331.410 4836675.748)
28909,14025083,21358,Parliament St,MNRML,Minor-Multi Level,2,501300,Minor,0,0.0,None,0.0,None,315195.161,4836623.547,-79.370895,43.671484,12102.0,POINT (631331.410 4836675.748)


In [43]:
int_min_min.to_file(simplify_dir + 'int_min_min.geojson', driver = 'GeoJSON')

## Minor/Minor Capture

In [18]:
min_min_capture = gpd.read_file(simplify_dir + 'int_min_min_capture.geojson')
stops_4 = stops_3[~stops_3['stop_id'].isin(list(min_min_capture['stop_id'].dropna()))]
stops_4.to_csv(simplify_dir + 'stops_no-stn-maj-majmin-minmin.csv', index = False)
stops_4

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding
2,265,1871,DAVISVILLE AVE AT CLEVELAND ST,NaN,43.702096,-79.378146,NaN,NaN,NaN,NaN,1.0
3,266,11700,DISCO RD AT ATTWELL DR,NaN,43.701331,-79.594777,NaN,NaN,NaN,NaN,1.0
4,267,3478,DISCO RD AT ATTWELL DR,NaN,43.701057,-79.595799,NaN,NaN,NaN,NaN,1.0
5,268,3479,DISCO RD AT CARLINGVIEW DR EAST SIDE,NaN,43.699820,-79.600767,NaN,NaN,NaN,NaN,1.0
6,269,3480,DISCO RD AT CARLINGVIEW DR,NaN,43.699893,-79.601112,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
10470,15365,S_SUKI,KING & SUMACH,NaN,43.655575,-79.358402,NaN,NaN,NaN,NaN,2.0
10471,15370,15526,DON MILLS RD AT BARBER GREEN SOUTH SIDE,NaN,43.727325,-79.340877,NaN,NaN,NaN,NaN,1.0
10476,15377,S_FIBA,FINCH AVE E AT BALDOON RD,NaN,43.815237,-79.232350,NaN,NaN,NaN,NaN,2.0
10477,15378,S_YKYK,YORK UNIVERSITY BUSWAY AT YORK BLVD,NaN,43.774503,-79.496783,NaN,NaN,NaN,NaN,2.0


# Finding Intersections Between Major Arterial and Collectors

In [24]:
centreline_maj_col = centreline[centreline['FEATURE_CODE_DESC'].isin(['Major Arterial', 'Collector'])]
int_maj_col = centreline_maj_col.merge(intersections, right_on = ['INT_ID'], left_on = ['FROM_INTERSECTION_ID']).append(
    (centreline_maj_col.merge(intersections, right_on = ['INT_ID'], left_on = ['TO_INTERSECTION_ID'])))

In [25]:
int_maj_col = int_maj_col[['INT_ID', 'FEATURE_CODE_DESC']]
int_maj_col = pd.get_dummies(int_maj_col, columns = ['FEATURE_CODE_DESC']).groupby('INT_ID').sum().reset_index()
int_maj_col = int_maj_col[(int_maj_col['FEATURE_CODE_DESC_Major Arterial']>0) & 
            (int_maj_col['FEATURE_CODE_DESC_Collector']>0)]

In [26]:
int_maj_col = intersections_geo[intersections_geo['INT_ID'].isin(list(int_maj_col['INT_ID']))]
int_maj_col = int_maj_col.to_crs(epsg = '32617')


In [48]:
int_maj_col.to_file(simplify_dir + 'int_maj_col.geojson', driver = 'GeoJSON')

## Major/Collector Capture

In [19]:
maj_col_capture = gpd.read_file(simplify_dir + 'int_maj_col_capture.geojson')
stops_5 = stops_4[~stops_4['stop_id'].isin(list(maj_col_capture['stop_id'].dropna()))]
stops_5.to_csv(simplify_dir + 'stops_no-stn-maj-majmin-minmin-majcol.csv', index = False)
stops_5

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding
2,265,1871,DAVISVILLE AVE AT CLEVELAND ST,NaN,43.702096,-79.378146,NaN,NaN,NaN,NaN,1.0
3,266,11700,DISCO RD AT ATTWELL DR,NaN,43.701331,-79.594777,NaN,NaN,NaN,NaN,1.0
4,267,3478,DISCO RD AT ATTWELL DR,NaN,43.701057,-79.595799,NaN,NaN,NaN,NaN,1.0
15,284,5006,DONCASTER AVE AT MAIN ST WEST SIDE,NaN,43.691661,-79.303783,NaN,NaN,NaN,NaN,1.0
16,285,5007,DONCASTER AVE AT MAIN ST,NaN,43.691625,-79.303534,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
10465,15355,15461,CHERRY ST AT FRONT ST EAST,NaN,43.652956,-79.358121,NaN,NaN,NaN,NaN,2.0
10466,15356,15462,DISTILLERY LOOP,NaN,43.650749,-79.356638,NaN,NaN,NaN,NaN,2.0
10469,15364,S_CNWE,WESTON RD & CANADA DRIVE,NaN,43.855517,-79.563041,NaN,NaN,NaN,NaN,2.0
10476,15377,S_FIBA,FINCH AVE E AT BALDOON RD,NaN,43.815237,-79.232350,NaN,NaN,NaN,NaN,2.0


# Important Note

During the last snapping, the capture file for `int_remaining` overwrote to the buffer file. If redoing the capture, remake the buffer file by deleting excess fields and removing duplicate geomteries.

In [27]:
int_remaining = intersections_geo[intersections_geo['CLASSIFI6'].isin(['MNRSL', 'MNRML'])]

In [28]:
int_remaining = int_remaining[~int_remaining['INT_ID'].isin(list(int_maj_min['INT_ID']))]
int_remaining = int_remaining[~int_remaining['INT_ID'].isin(list(int_maj_col['INT_ID']))]
int_remaining = int_remaining[~int_remaining['INT_ID'].isin(list(int_min_min['INT_ID']))]

In [31]:
int_remaining = int_remaining[['INT_ID', 'INTERSEC5', 'geometry']].drop_duplicates()
int_remaining = int_remaining.to_crs(epsg = '32617')
int_remaining.to_file(simplify_dir + 'int_remaining_minor.geojson', driver = 'GeoJSON')

## Filtering Minor Intersections

In [58]:
shape_int_geo = gpd.read_file(simplify_dir + 'shape_int.geojson')

In [59]:
shape_int = shape_int_geo[['INT_ID']].drop_duplicates()

In [60]:
shape_int = shape_int[['INT_ID']].dropna()

In [61]:
shape_int['INT_ID'] = shape_int['INT_ID'].astype(int)

In [64]:
int_remaining = int_remaining[int_remaining['INT_ID'].isin(list(shape_int['INT_ID']))]

In [65]:
int_remaining.to_file(simplify_dir + 'int_remaining_minor.geojson', driver = 'GeoJSON')